In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import datetime

import torch
import torch.nn as nn
import torch.utils.data as Data
# If pytorch 0.3.1, update by adding "soumath" in the path and run "conda update pytorch" in Anaconda
#print(torch.__version__)

C:\Users\LEN\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
CIFAR = h5py.File("..\CIFAR10.hdf5","r")
x_train = np.float32(CIFAR['X_train'][:])
x_test = np.float32(CIFAR['X_test'][:])
y_train = np.int32(CIFAR['Y_train'][:])
y_test = np.int32(CIFAR['Y_test'][:])

In [3]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((50000, 3, 32, 32), (10000, 3, 32, 32), (50000,), (10000,))

In [5]:
# Transform into torch.Tensor
x_traints = torch.Tensor(x_train)
y_traints = torch.Tensor(y_train)
x_testts = torch.Tensor(x_test)
y_testts = torch.Tensor(y_test)

In [6]:
# Pack as torch.Dataset
trainset = Data.TensorDataset(x_traints, y_traints)
testset = Data.TensorDataset(x_testts, y_testts)
# Put datasets into Data Loader
BatchSize = 100
train_loader = Data.DataLoader(dataset=trainset, batch_size=BatchSize, shuffle=True)
test_loader = Data.DataLoader(dataset=testset, batch_size=BatchSize, shuffle=False)

In [ ]:
# Data augmentation
# See: https://pytorch.org/docs/stable/torchvision/transforms.html
# Also see: https://blog.csdn.net/u014380165/article/details/79167753
from torchvision import transforms
ds_trans = transforms.Compose([transforms.RandomCrop(32),
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomVerticalFlip(),
                              transforms.RandomRotation(degrees=45),
                              transforms.ColorJitter(
                                brightness=0.1*torch.randn(1),
                                contrast=0.1*torch.randn(1),
                                saturation=0.1*torch.randn(1),
                                hue=0.1*torch.randn(1))])

In [7]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
# Create torch model with structure given in Slides "Lecture 6"
LongConv = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=64,kernel_size=4,stride=1,padding=2),
                         nn.ReLU(inplace=True), # Conv layer 1
                         nn.BatchNorm2d(num_features=64),
                         nn.Conv2d(64,64,4,padding=2),
                         nn.ReLU(inplace=True), # Conv layer 2
                         nn.MaxPool2d(kernel_size=2,stride=2), # Max Pooling
                         nn.Dropout2d(p=0.25), # Dropout
                         nn.Conv2d(64,64,4,padding=2), 
                         nn.ReLU(inplace=True), # Conv layer 3
                         nn.BatchNorm2d(64),
                         nn.Conv2d(64,64,4,padding=2),
                         nn.ReLU(inplace=True), # Conv layer 4
                         nn.MaxPool2d(kernel_size=2,stride=2), # Max Pooling
                         nn.Dropout2d(p=0.25), # Dropout
                         nn.Conv2d(64,64,4,padding=2), 
                         nn.ReLU(inplace=True), # Conv layer 5
                         nn.BatchNorm2d(64),
                         nn.Conv2d(64,64,3),
                         nn.ReLU(inplace=True), # Conv layer 6
                         nn.Dropout2d(p=0.25), # Dropout
                         nn.Conv2d(64,64,3),
                         nn.ReLU(inplace=True), # Conv layer 7
                         nn.BatchNorm2d(64),
                         nn.Conv2d(64,64,3),
                         nn.ReLU(inplace=True), # Conv layer 8
                         nn.BatchNorm2d(64),
                         nn.Dropout2d(p=0.25)) # Dropout
Classifier = nn.Sequential(
                         nn.Linear(in_features=64*4*4,out_features=500,bias=True),
                         #nn.Linear(in_features=500,out_features=500,bias=True),
                         nn.Linear(in_features=500,out_features=10,bias=True),
                         nn.Softmax(dim=1))

In [9]:
# Define a Convolution NN class
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.ConvLayer = LongConv
        self.LinearLayer = Classifier
        
    def forward(self,x):
        ConvOut = self.ConvLayer(x)
        Out = ConvOut.reshape(ConvOut.shape[0],-1)
        ClassOut = self.LinearLayer(Out)
        return ClassOut

In [18]:
# Define model
model = ConvNet().to(device)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)

In [22]:
# Small Data loader as a local train set
x_trainsmallts = torch.Tensor(x_train[:2000, :])
y_trainsmallts = torch.Tensor(y_train[:2000])
trainsmallset = Data.TensorDataset(x_trainsmallts, y_trainsmallts)
smallBatchSize = 10
trainsmall_loader = Data.DataLoader(dataset=trainsmallset, batch_size=smallBatchSize, shuffle=True)

In [23]:
# Training process
start_time = datetime.datetime.now()
num_epochs = 10
num_steps = len(train_loader)
#num_steps = len(trainsmall_loader)
total = 0
correct = 0

for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
    #for i,(images,labels) in enumerate(trainsmall_loader):
        images = images.to(device)
        labels = labels.to(device).long()
            
        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (labels == predicted).sum().item()
            
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        if (i+1)%100 == 0:
            print("Epoch[{}/{}], Step[{}/{}], Loss {:4f}, Accuracy {:4f}%".format(
                epoch+1, num_epochs, i+1, num_steps, loss.item(), correct/total*100))
            now_time = datetime.datetime.now()
            print("Total cost time:{}".format(now_time-start_time))

Epoch[1/10], Step[100/200], Loss 1.929070, Accuracy 48.000000%
Total cost time:0:00:32.724475
Epoch[1/10], Step[200/200], Loss 1.882686, Accuracy 49.300000%
Total cost time:0:01:33.327817
Epoch[2/10], Step[100/200], Loss 2.117649, Accuracy 49.766667%
Total cost time:0:02:24.424939
Epoch[2/10], Step[200/200], Loss 2.063555, Accuracy 50.150000%
Total cost time:0:03:11.109134
Epoch[3/10], Step[100/200], Loss 1.812820, Accuracy 50.440000%
Total cost time:0:03:55.628519
Epoch[3/10], Step[200/200], Loss 1.993241, Accuracy 50.500000%
Total cost time:0:04:42.173840
Epoch[4/10], Step[100/200], Loss 1.994628, Accuracy 50.657143%
Total cost time:0:05:12.816984
Epoch[4/10], Step[200/200], Loss 2.040266, Accuracy 50.950000%
Total cost time:0:05:45.077648
Epoch[5/10], Step[100/200], Loss 1.856857, Accuracy 51.166667%
Total cost time:0:06:15.663842
Epoch[5/10], Step[200/200], Loss 1.701901, Accuracy 51.260000%
Total cost time:0:06:44.980433
Epoch[6/10], Step[100/200], Loss 2.018041, Accuracy 51.45454

In [24]:
# Evaluate with test set
model.eval()
with torch.no_grad():
    correct_test = 0
    total_test = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device).long()
        
        outputs = model(images)
        loss_test = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (labels == predicted).sum().item()
    
    print("Test Accuracy of the model on test images:{}%".format(correct_test / total_test * 100))

Test Accuracy of the model on test images:52.82%


In [25]:
# Save model
torch.save(model,"ConvModel.ckpt")

C:\Users\LEN\Anaconda3\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type ConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
